In [6]:
sc

AttributeError: 'NoneType' object has no attribute 'sc'

<SparkContext master=local[*] appName=PySparkShell>

In [3]:
spark.stop()

### Reading data from a JDBC source

In [7]:
from pyspark.sql import SparkSession
mysql_connector_path = "/home/hadoop/Downloads/mysql-connector-java-8.0.13.jar"

In [8]:
spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()

In [5]:
# restart 
#pyspark --jars "path of jar package"

In [9]:
hremployeeDF = spark.read.format("jdbc")\
    .option("url", "jdbc:mysql://localhost:3306/hremployeeDB")\
    .option("dbtable", "hremployee").option("user", "root").option("password", "hadoop@123")\
    .option("driver", "com.mysql.cj.jdbc.Driver").load()

In [10]:
hremployeeDF.createOrReplaceTempView("hrtable")

In [11]:
hremployeeDF.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(hremployee) [numPartitions=1] [EmployeeID#0,Department#1,JobRole#2,Attrition#3,Gender#4,Age#5,MaritalStatus#6,Education#7,EducationField#8,BusinessTravel#9,JobInvolvement#10,JobLevel#11,JobSatisfaction#12,Hourlyrate#13,Income#14,Salaryhike#15,OverTime#16,Workex#17,YearsSinceLastPromotion#18,EmpSatisfaction#19,TrainingTimesLastYear#20,WorkLifeBalance#21,Performance_Rating#22] PushedFilters: [], ReadSchema: struct<EmployeeID:string,Department:string,JobRole:string,Attrition:string,Gender:string,Age:int,...


#### 1.Display The shape of hrtable

In [10]:
len(hr_df.columns) #colums

23

In [11]:
hr_df.count() #rows

1469

In [12]:
# rows
spark.sql("select count(*) as rows from hrtable").show()

+--------+
|count(1)|
+--------+
|    1469|
+--------+



In [14]:
# columns
columns = spark.sql("""describe hrtable""").collect()
len(columns)

23

hr_df.printSchema()

#### write query to show first 3 employee from each job roles to join the company

In [30]:
spark.sql("""
            select * from (
            select EmployeeID,Department,JobRole, row_number() over(partition by JobRole
            order by EmployeeID) as row_num from hrtable) 
            where row_num<4""").show()

+----------+--------------------+--------------------+-------+
|EmployeeID|          Department|             JobRole|row_num|
+----------+--------------------+--------------------+-------+
|         1|               Sales|     Sales Executive|      1|
|      1012|               Sales|     Sales Executive|      2|
|      1014|               Sales|     Sales Executive|      3|
|      1003|Research & Develo...|Manufacturing Dir...|      1|
|      1034|Research & Develo...|Manufacturing Dir...|      2|
|      1038|Research & Develo...|Manufacturing Dir...|      3|
|       100|Research & Develo...|Laboratory Techni...|      1|
|      1001|Research & Develo...|Laboratory Techni...|      2|
|      1002|Research & Develo...|Laboratory Techni...|      3|
|      1013|               Sales|Sales Representative|      1|
|      1022|               Sales|Sales Representative|      2|
|      1057|               Sales|Sales Representative|      3|
|        10|Research & Develo...|Healthcare Repres...| 

#### 3. write a query to show top3 employee from each job role earning maximum salary 

In [35]:
spark.sql("""
            select * from (
            select EmployeeID,Department,JobRole,Income,rank() over(partition by JobRole
            order by Income desc) as rank_num from hrtable) 
            where rank_num<4""").show()

+----------+--------------------+--------------------+------+--------+
|EmployeeID|          Department|             JobRole|Income|rank_num|
+----------+--------------------+--------------------+------+--------+
|        99|               Sales|     Sales Executive| 13872|       1|
|       545|               Sales|     Sales Executive| 13770|       2|
|       839|               Sales|     Sales Executive| 13758|       3|
|       722|Research & Develo...|Manufacturing Dir...| 13973|       1|
|       628|Research & Develo...|Manufacturing Dir...| 13826|       2|
|       744|Research & Develo...|Manufacturing Dir...| 13726|       3|
|       678|Research & Develo...|Laboratory Techni...|  7403|       1|
|       817|Research & Develo...|Laboratory Techni...|  6782|       2|
|       945|Research & Develo...|Laboratory Techni...|  6674|       3|
|       565|               Sales|Sales Representative|  6632|       1|
|      1308|               Sales|Sales Representative|  5405|       2|
|     

In [15]:
hremployeeDF.columns

['EmployeeID',
 'Department',
 'JobRole',
 'Attrition',
 'Gender',
 'Age',
 'MaritalStatus',
 'Education',
 'EducationField',
 'BusinessTravel',
 'JobInvolvement',
 'JobLevel',
 'JobSatisfaction',
 'Hourlyrate',
 'Income',
 'Salaryhike',
 'OverTime',
 'Workex',
 'YearsSinceLastPromotion',
 'EmpSatisfaction',
 'TrainingTimesLastYear',
 'WorkLifeBalance',
 'Performance_Rating']

#### 4. show top3 highest package for overall job role

In [37]:
spark.sql("""
            select * from (
            select EmployeeID,Department,JobRole,Income,rank() over(
            order by Income desc) as rank_num from hrtable) 
            where rank_num<4""").show()

+----------+--------------------+-----------------+------+--------+
|EmployeeID|          Department|          JobRole|Income|rank_num|
+----------+--------------------+-----------------+------+--------+
|       191|Research & Develo...|          Manager| 19999|       1|
|       747|Research & Develo...|Research Director| 19973|       2|
|       852|Research & Develo...|          Manager| 19943|       3|
+----------+--------------------+-----------------+------+--------+



#### 5. Dense Rank()

In [45]:
spark.sql("""
        select employeeid, department,jobrole,age, gender, income,workex,
        dense_rank() over(partition by JobRole order by age, workex) as rank_by_age 
        from hrtable
        """).show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|rank_by_age|
+----------+----------+---------------+---+------+------+------+-----------+
|        97|     Sales|Sales Executive| 24|Female|  4999|     4|          1|
|       526|     Sales|Sales Executive| 24|Female|  4577|     4|          1|
|       381|     Sales|Sales Executive| 24|Female|  4260|     5|          2|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|          2|
|       518|     Sales|Sales Executive| 25|  Male|  4851|     4|          3|
|       398|     Sales|Sales Executive| 25|Female|  4487|     5|          4|
|       635|     Sales|Sales Executive| 25|  Male|  4194|     5|          4|
|       639|     Sales|Sales Executive| 25|  Male|  4256|     5|          4|
|       886|     Sales|Sales Executive| 25|  Male|  4950|     5|          4|
|       108|     Sales|Sales Executive| 25|  Male|  5744|     6|          5|

#### 6.Lag()
    - Is a window function that allows you access records from previous row
    
    - It will be useful in comparing records between rows or calculation differences 
    - lag(column_name, offset, default)

In [48]:
spark.sql("""
        select employeeid, department,jobrole,age, gender,income, lag(income,2,0)
        over(partition by JobRole order by employeeid) as previous_income 
        from hrtable
        """).show()

+----------+----------+---------------+---+------+------+---------------+
|employeeid|department|        jobrole|age|gender|income|previous_income|
+----------+----------+---------------+---+------+------+---------------+
|         1|     Sales|Sales Executive| 41|Female|  5993|              0|
|      1012|     Sales|Sales Executive| 36|Female|  9278|              0|
|      1014|     Sales|Sales Executive| 30|Female|  4779|           5993|
|      1020|     Sales|Sales Executive| 36|Female|  5647|           9278|
|      1026|     Sales|Sales Executive| 24|Female|  4162|           4779|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|           5647|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|           4162|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|           9204|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|          10793|
|      1042|     Sales|Sales Executive| 28|  Male|  8463|          10096|
|      1048|     Sales|Sales Executive

### 7. NTILE()
    - Dividing records into quantile

In [52]:
spark.sql("""
        select employeeid, department,jobrole,age, gender,income, NTILE(4)
        over(order by Income) as salary_quantiler 
        from hrtable
        """).show(500)

+----------+--------------------+--------------------+---+------+------+----------------+
|employeeid|          department|             jobrole|age|gender|income|salary_quantiler|
+----------+--------------------+--------------------+---+------+------+----------------+
|       514|Research & Develo...|  Research Scientist| 20|  Male|  1009|               1|
|       728|Research & Develo...|  Research Scientist| 18|  Male|  1051|               1|
|       765|               Sales|Sales Representative| 28|  Male|  1052|               1|
|      1338|               Sales|Sales Representative| 30|  Male|  1081|               1|
|      1365|               Sales|Sales Representative| 29|  Male|  1091|               1|
|       178|Research & Develo...|Laboratory Techni...| 19|  Male|  1102|               1|
|       912|               Sales|Sales Representative| 25|  Male|  1118|               1|
|      1402|Research & Develo...|Laboratory Techni...| 31|Female|  1129|               1|
|       30

#### calculate max, mean and average income from each quantile

In [65]:
# mean salary, median, maximum
spark.sql("""
        select salary_quantile,mean(Income),max(Income),percentile_approx(Income,0.5) as median from
        (select employeeid, department, jobrole, age, gender,income, NTILE(4)
        over(order by Income) as salary_quantile 
        from hrtable) group by salary_quantile
        """).show()

+---------------+------------------+-----------+------+
|salary_quantile|       avg(Income)|max(Income)|median|
+---------------+------------------+-----------+------+
|              1|2353.9972826086955|       2911|  2377|
|              2|3966.7356948228885|       4930|  4033|
|              3| 6191.138964577656|       8380|  6091|
|              4|13522.326975476839|      19999| 13120|
+---------------+------------------+-----------+------+



### 8.Percent_Rank()
    -Window function that calculates relative rank() of row within a result set expressed as percentage

In [66]:
spark.sql("""
        select employeeid, department,jobrole,income,workex,
        percent_rank() over(partition by department order by income desc, workex) as percent_rank
        from hrtable
        """).show()

+----------+----------+-------+---+------+------+------+--------------------+
|employeeid|department|jobrole|age|gender|income|workex|        percent_rank|
+----------+----------+-------+---+------+------+------+--------------------+
|       919|     Sales|Manager| 51|  Male| 19847|    31|                 0.0|
|       750|     Sales|Manager| 52|Female| 19845|    33|0.002252252252252...|
|      1242|     Sales|Manager| 40|  Male| 19833|    21|0.004504504504504...|
|      1116|     Sales|Manager| 55|  Male| 19586|    36|0.006756756756756757|
|       234|     Sales|Manager| 50|Female| 19517|    32|0.009009009009009009|
|      1126|     Sales|Manager| 50|  Male| 19331|    27| 0.01126126126126126|
|       238|     Sales|Manager| 52|  Male| 19068|    33|0.013513513513513514|
|        30|     Sales|Manager| 46|Female| 18947|    22|0.015765765765765764|
|       914|     Sales|Manager| 45|Female| 18824|    26|0.018018018018018018|
|       917|     Sales|Manager| 46|Female| 18789|    26| 0.02027

#### Find the total employees in each percentile group 0-25, 25-50, 50-75, 75-100

In [103]:
q1 = spark.sql("""      
        select case 
        when percent_rank <=0.25 then 'Q1'
        when percent_rank <=0.50 then 'Q2'
        when percent_rank <=0.75 then 'Q3'
        else 'Q4'
        end as perc_range, count(*),avg(percent_rank) as average_percent
        from (select employeeid, department,jobrole,income,workex,
        percent_rank() over(partition by department order by income desc, workex) as percent_rank
        from hrtable)
        group by perc_range
        order by perc_range desc
        """).show()

+----------+--------+-------------------+
|perc_range|count(1)|    average_percent|
+----------+--------+-------------------+
|        Q4|     367| 0.8758451785924978|
|        Q3|     366| 0.6258331631141983|
|        Q2|     367|0.37585415384840765|
|        Q1|     369|0.12481905587727776|
+----------+--------+-------------------+



In [94]:
q1 = spark.sql("""      
        select case 
        when percent_rank <=0.25 then 'Q1'
        when percent_rank <=0.50 then 'Q2'
        when percent_rank <=0.75 then 'Q3'
        else 'Q4'
        end as perc_range
        from (select employeeid, department,jobrole,income,workex,
        percent_rank() over(partition by department order by income desc, workex) as percent_rank
        from hrtable)
        group by perc_range
        """).show()

+----------+
|perc_range|
+----------+
|        Q2|
|        Q1|
|        Q3|
|        Q4|
+----------+



#### show jobrole with maximum Attrition Rate in percentage

In [135]:
spark.sql("""
    select count(jobrole)case
    when attrition =='Yes' then 1
    else 0
    end as Attrition
    from hrtable 
    
    """).show()

AnalysisException: "grouping expressions sequence is empty, and 'hrtable.`jobrole`' is not an aggregate function. Wrap '(sum(CAST(CASE WHEN (hrtable.`attrition` = 'Yes') THEN 1 ELSE 0 END AS BIGINT)) AS `Attrition`)' in windowing function(s) or wrap 'hrtable.`jobrole`' in first() (or first_value) if you don't care which value you get.;;\nAggregate [jobrole#2, sum(cast(CASE WHEN (attrition#3 = Yes) THEN 1 ELSE 0 END as bigint)) AS Attrition#2873L]\n+- SubqueryAlias `hrtable`\n   +- Relation[EmployeeID#0,Department#1,JobRole#2,Attrition#3,Gender#4,Age#5,MaritalStatus#6,Education#7,EducationField#8,BusinessTravel#9,JobInvolvement#10,JobLevel#11,JobSatisfaction#12,Hourlyrate#13,Income#14,Salaryhike#15,OverTime#16,Workex#17,YearsSinceLastPromotion#18,EmpSatisfaction#19,TrainingTimesLastYear#20,WorkLifeBalance#21,Performance_Rating#22] JDBCRelation(hremployee) [numPartitions=1]\n"

In [140]:
spark.sql(""" 
    select count(*),jobrole,
    sum(case when attrition='Yes' then 1 else 0 end) as rate,
    (sum(case when attrition='Yes' then 1 else 0 end)*100/count(*)) as percentage
    from hrtable group by jobrole order by percentage desc
    
""").show()

+--------+--------------------+----+------------------+
|count(1)|             jobrole|rate|        percentage|
+--------+--------------------+----+------------------+
|      82|Sales Representative|  33| 40.24390243902439|
|     259|Laboratory Techni...|  62| 23.93822393822394|
|      52|     Human Resources|  12|23.076923076923077|
|     326|     Sales Executive|  57|17.484662576687118|
|     292|  Research Scientist|  47|16.095890410958905|
|     145|Manufacturing Dir...|  10| 6.896551724137931|
|     131|Healthcare Repres...|   9| 6.870229007633588|
|     102|             Manager|   5| 4.901960784313726|
|      80|   Research Director|   2|               2.5|
+--------+--------------------+----+------------------+



## Hive Integration with pyspark

In [1]:
spark.stop()

In [2]:
#Spark Integration with Hive with Spark Session
spark = (SparkSession.builder.appName("pyspark-hive-integration")
        .config('spark.sql.warehouse.dir','/user/hive/warehouse/')
         .enableHiveSupport().getOrCreate())

In [3]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|    airlines|
|  banking_db|
|     default|
+------------+



In [4]:
spark.sql("create database if not exists airlines").show()

++
||
++
++



In [17]:
spark.sql("""
            create table if not exists airlines.flights(DayofMonth int,
            DayOfWeek int, Carrier varchar(10),OriginAirportID int,
            DestAirportID int,DepDelay int, ArrDelay int)
            row format delimited
            fields terminated by ','
            lines terminated by '\n'
            STORED AS TEXTFILE 
            TBLPROPERTIES ('skip.header.line.count'='1')
    
            """)

DataFrame[]

In [4]:
spark.sql("use airlines")

DataFrame[]

In [19]:
spark.sql("describe flights").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     DayofMonth|      int|   null|
|      DayOfWeek|      int|   null|
|        Carrier|   string|   null|
|OriginAirportID|      int|   null|
|  DestAirportID|      int|   null|
|       DepDelay|      int|   null|
|       ArrDelay|      int|   null|
+---------------+---------+-------+



In [5]:
#overwrite into table flights
spark.sql("""
          load data local inpath '/home/hadoop/Downloads/raw_flight_data1.csv'
          overwrite into table flights""")

DataFrame[]

In [7]:
spark.sql("select * from flights limit 1").show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
+----------+---------+-------+---------------+-------------+--------+--------+



In [8]:
# create table airport
spark.sql("""
            create table if not exists airlines.airport(airport_id int,
            city string, state varchar(10),airport_name string)
            row format delimited
            fields terminated by ','
            lines terminated by '\n'
            STORED AS TEXTFILE 
            TBLPROPERTIES ('skip.header.line.count'='1')
    
            """)

DataFrame[]

In [9]:
#overwrite into table airport
spark.sql("""
          load data local inpath '/home/hadoop/Downloads/airports1.csv'
          overwrite into table airport""")

DataFrame[]

In [11]:
spark.sql("select * from airport limit 1").show()

+----------+-----------+-----+------------+
|airport_id|       city|state|airport_name|
+----------+-----------+-----+------------+
|     10165|Adak Island|   AK|        Adak|
+----------+-----------+-----+------------+



In [13]:
flight_df = spark.table("flights")
airport_df = spark.table("airport")

#### Transformation

In [14]:
combined_df = flight_df.join(airport_df, on = airport_df.airport_id == flight_df.OriginAirportID, how="inner")

In [15]:
combined_df.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|        airport_name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
only showing top 2 rows



In [16]:
combined_left_df = flight_df.join(airport_df, on = airport_df.airport_id == flight_df.OriginAirportID, how="left")

In [17]:
combined_left_df.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|        airport_name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
only showing top 2 rows



In [19]:
combined_right_df = flight_df.join(airport_df, on = airport_df.airport_id == flight_df.OriginAirportID, how="right")

In [20]:
combined_right_df.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|city|state|        airport_name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|        19|        5|     DL|          14570|        14869|      -5|      -8|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        19|        5|     DL|          14570|        14869|      -4|       2|     14570|Reno|   NV|Reno/Tahoe Intern...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
only showing top 2 rows



In [21]:
#A-B
combined_leftanti_df = flight_df.join(airport_df, on = airport_df.airport_id == flight_df.OriginAirportID, how="leftanti")

In [22]:
combined_leftanti_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
+----------+---------+-------+---------------+-------------+--------+--------+



In [23]:
#only print left table which is common in right side
combined_leftsemi_df = flight_df.join(airport_df, on = airport_df.airport_id == flight_df.OriginAirportID, how="leftsemi")

In [24]:
combined_leftsemi_df.show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [ ]:
combined_df.repartition()

In [26]:
combined_df.write.partitionBy('Carrier').parquet("/airlines")

In [33]:
airline_join = spark.read.parquet('/airlines')

In [34]:
airline_join.createOrReplaceTempView("airline")

In [37]:
spark.sql("select avg(ArrDelay) from airline where Carrier='AA' ").show()

+------------------+
|     avg(ArrDelay)|
+------------------+
|11.651327332054372|
+------------------+



In [38]:
spark.sql("describe airline").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     DayofMonth|      int|   null|
|      DayOfWeek|      int|   null|
|OriginAirportID|      int|   null|
|  DestAirportID|      int|   null|
|       DepDelay|      int|   null|
|       ArrDelay|      int|   null|
|     airport_id|      int|   null|
|           city|   string|   null|
|          state|   string|   null|
|   airport_name|   string|   null|
|        Carrier|   string|   null|
+---------------+---------+-------+



In [39]:
combined_df.columns

['DayofMonth',
 'DayOfWeek',
 'Carrier',
 'OriginAirportID',
 'DestAirportID',
 'DepDelay',
 'ArrDelay',
 'airport_id',
 'city',
 'state',
 'airport_name']

## Hbase Integration with Pyspark

In [2]:
spark.stop()

In [6]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HbaseIntegration").getOrCreate()

In [7]:
hbase_conf = {
    "hbase.zookeeper.quorum":'localhost',
    "hbase.mapreduce.inputtable":'hcustomer',
    "hbase.client.scanner.timeout.period":"600000"
}

In [ ]:
spark.read.format("org.apache.hadoop.hbase.spark").options(**hbase_conf).load()

## Load on MySQL

In [40]:
# fetch the combined_df data into mysql 
connection_properties = {
    "user":"root",
    "password":"hadoop@123",
    "driver":"com.mysql.cj.jdbc.Driver"
}
combined_df.write.jdbc(url="jdbc:mysql://localhost:3306/airlines",table="airline",
                mode = "overwrite",properties = connection_properties)